In [466]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv("master_data_Marian.20251020.csv")
df

,ID,REG_TERM,CLASS_LEVEL,COLL1,COLL2,MAJOR1,MAJOR2,MAJOR3,MAJOR4,MINOR1,MINOR2,MINOR3,MINOR4,FirstTerm_BM,LastTerm_BM,Student_Status,FIN221_IND,ACCY200_IND,ACCY201_IND,ACCY202_IND,BADM310_IND,BADM320_IND,GradTerm,BM_Grad_IND,BM_Declaration_IND,BM_Drop_IND,Completion,Core_Completion,Elective_Completion
0,48804591,120235,Senior,KM,,Finance,,,,,,,,NaN,NaN,Gies,0,0,0,0,0,0,NaN,2,0,0,5,3,2
1,96256798,120248,Freshman,KP,,Aerospace Engineering,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
2,49203353,120241,Senior,KY,,Interdisciplinary Health Sci,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,120241.0,0,0,0,0,0,0
3,15838848,120221,Senior,KV,,Statistics,Economics,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
4,21586953,120231,Senior,KR,KL,Instrumental Music,,Animal Sciences,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354632,34344538,120228,Sophomore,KV,,Biology,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
354633,55271473,120221,Junior,KP,,"Nuclear, Plasma, Radiolgc Engr",,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
354634,97720368,120258,Senior,KV,,Psychology,,,,"Criminology, Law, and Society",Legal Studies,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
354635,29013981,120218,Sophomore,KM,,Business Undeclared,,,,,,,,NaN,NaN,Gies,0,0,0,0,0,0,NaN,2,0,0,0,0,0


Step 1: Remove Business Majors 

In [467]:
college_to_remove1 = "KM"
df_filter = df[df["COLL1"] != college_to_remove1]
df_filter = df_filter[df_filter["COLL2"] !=college_to_remove1]
df_filter

,ID,REG_TERM,CLASS_LEVEL,COLL1,COLL2,MAJOR1,MAJOR2,MAJOR3,MAJOR4,MINOR1,MINOR2,MINOR3,MINOR4,FirstTerm_BM,LastTerm_BM,Student_Status,FIN221_IND,ACCY200_IND,ACCY201_IND,ACCY202_IND,BADM310_IND,BADM320_IND,GradTerm,BM_Grad_IND,BM_Declaration_IND,BM_Drop_IND,Completion,Core_Completion,Elective_Completion
1,96256798,120248,Freshman,KP,,Aerospace Engineering,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
2,49203353,120241,Senior,KY,,Interdisciplinary Health Sci,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,120241.0,0,0,0,0,0,0
3,15838848,120221,Senior,KV,,Statistics,Economics,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
4,21586953,120231,Senior,KR,KL,Instrumental Music,,Animal Sciences,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
5,18069848,120241,Senior,KY,,Community Health,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354631,83474981,120221,Junior,KL,,Agr & Consumer Economics,,,,Business,Leadership Studies,,,120221.0,120241.0,BM-NEW,0,0,0,0,0,0,NaN,2,1,0,0,0,0
354632,34344538,120228,Sophomore,KV,,Biology,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
354633,55271473,120221,Junior,KP,,"Nuclear, Plasma, Radiolgc Engr",,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0
354634,97720368,120258,Senior,KV,,Psychology,,,,"Criminology, Law, and Society",Legal Studies,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0


In [468]:
# Add per-student previous-term comparisons and four columns on df_filter
df_filter = df_filter.copy()

# Normalize REG_TERM and build a simple chronological index from df_filter

df_filter['REG_TERM_str'] = df_filter['REG_TERM'].astype(str).str.strip().fillna('')
df_filter['REG_TERM_str'] = df_filter['REG_TERM_str'].str.replace(r'\.0+$', '', regex=True)
df_filter['term_code'] = pd.to_numeric(df_filter['REG_TERM_str'], errors='coerce').astype('Int64')

term_order = sorted(df_filter['term_code'].dropna().unique())

term_index_map = {v: i for i, v in enumerate(term_order)}

df_filter['term_index'] = df_filter['term_code'].map(term_index_map).astype('Int64')

# Ensure completion component columns exist and are numeric (fill missing with 0)
df_filter['Core_Completion'] = pd.to_numeric(df_filter.get('Core_Completion', pd.Series(dtype='float')), errors='coerce').fillna(0)
df_filter['Elective_Completion'] = pd.to_numeric(df_filter.get('Elective_Completion', pd.Series(dtype='float')), errors='coerce').fillna(0)
if 'Completion' in df_filter.columns:
    df_filter['Completion'] = pd.to_numeric(df_filter['Completion'], errors='coerce').fillna(0)
else:
    df_filter['Completion'] = (df_filter['Core_Completion'] + df_filter['Elective_Completion']).fillna(0)
# Sort so groupby.shift uses chronological order per student
df_filter = df_filter.sort_values(['ID', 'term_index']).reset_index(drop=True)
# Compute previous-term values, treating missing previous as 0 (per your request)
df_filter['prev_core'] = df_filter.groupby('ID')['Core_Completion'].shift(1).fillna(0)
df_filter['prev_elective'] = df_filter.groupby('ID')['Elective_Completion'].shift(1).fillna(0)
df_filter['prev_total'] = df_filter.groupby('ID')['Completion'].shift(1).fillna(0)
# Compute numeric increases
df_filter['core_increase'] = df_filter['Core_Completion'] - df_filter['prev_core']
df_filter['elective_increase'] = df_filter['Elective_Completion'] - df_filter['prev_elective']
df_filter['total_increase'] = df_filter['Completion'] - df_filter['prev_total']
# Boolean 'taking or not' using total_increase > 0
df_filter['took_this_term'] = df_filter['total_increase'] > 0
# (Optional) convenience boolean columns for component increases
df_filter['core_increase_flag'] = df_filter['core_increase'] > 0
df_filter['elective_increase_flag'] = df_filter['elective_increase'] > 0
# Diagnostics: print totals and show a small sample
print('Rows total:', len(df_filter))
print('Took this term (total_increase>0):', int(df_filter['took_this_term'].sum()))
print('Core increase >0:', int(df_filter['core_increase_flag'].sum()))
print('Elective increase >0:', int(df_filter['elective_increase_flag'].sum()))
pd.set_option('display.max_rows', 20)
pd.options.display.max_columns = None
display(df_filter.head(50))

Rows total: 323267
Took this term (total_increase>0): 17697
Core increase >0: 14505
Elective increase >0: 5597


,ID,REG_TERM,CLASS_LEVEL,COLL1,COLL2,MAJOR1,MAJOR2,MAJOR3,MAJOR4,MINOR1,MINOR2,MINOR3,MINOR4,FirstTerm_BM,LastTerm_BM,Student_Status,FIN221_IND,ACCY200_IND,ACCY201_IND,ACCY202_IND,BADM310_IND,BADM320_IND,GradTerm,BM_Grad_IND,BM_Declaration_IND,BM_Drop_IND,Completion,Core_Completion,Elective_Completion,REG_TERM_str,term_code,term_index,prev_core,prev_elective,prev_total,core_increase,elective_increase,total_increase,took_this_term,core_increase_flag,elective_increase_flag
0,10000197,120218,Sophomore,KV,,General Curriculum,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120218,120218,0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
1,10000197,120220,Junior,KV,,General Curriculum,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120220,120220,1,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
2,10000197,120221,Junior,KV,,Molecular and Cellular Biology,,,,Informatics,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120221,120221,2,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
3,10000197,120228,Junior,KV,,Molecular and Cellular Biology,,,,Informatics,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120228,120228,5,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
4,10000197,120231,Senior,KV,,Molecular and Cellular Biology,,,,Informatics,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120231,120231,7,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,10010913,120221,Sophomore,KP,,Mechanical Engineering,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120221,120221,2,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
46,10010913,120228,Junior,KP,,Mechanical Engineering,,,,Computer Science,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120228,120228,5,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
47,10010913,120231,Junior,KP,,Mechanical Engineering,,,,Computer Science,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120231,120231,7,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
48,10010913,120238,Senior,KP,,Mechanical Engineering,,,,Computer Science,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120238,120238,10,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False


In [469]:
df_filter = df_filter[~df_filter['REG_TERM'].astype(str).str.endswith(('0','5','7'))]
df_filter
college_names = {
   'KP': 'Grainger College of Engineering (GCOE)',
   'KY': 'College of Applied Health Sciences (AHS)',
   'KV': 'College of Liberal Arts and Sciences (LAS)',
   'KR': 'College of Fine and Applied Arts (FAA)',
   'KL': 'College of Agricultural, Consumer & Environmental Sciences (ACES)',
   'LL': 'School of Social Work (SSW)',
   'KW': 'Division of Exploratory Studies (Undeclared)',
   'KT': 'College of Media',
   'LP': 'School of Information Sciences (iSchool)',
   'KN': 'College of Education (EDUC)',
   'KU': 'College of Law (LAW)',
   'LG': 'Graduate Students',
   'NB': 'Provost & VC Acad Affairs',
   'LN': 'Center Innov in Teach Learn'
}

term_dict = {
    120211: "2021-Spring",
    120218: "2021-Fall",
    120221: "2022-Spring",
    120228: "2022-Fall",
    120231: "2023-Spring",
    120238: "2023-Fall",
    120241: "2024-Spring",
    120248: "2024-Fall",
    120251: "2025-Spring",
    120258: "2025-Fall",
}
# df_filter['REG_TERM'] = df_filter['REG_TERM'].replace(term_dict)
df_filter['COLL1'] = df_filter['COLL1'].replace(college_names)
df_filter['COLL2'] = df_filter['COLL2'].replace(college_names)

df_filter

,ID,REG_TERM,CLASS_LEVEL,COLL1,COLL2,MAJOR1,MAJOR2,MAJOR3,MAJOR4,MINOR1,MINOR2,MINOR3,MINOR4,FirstTerm_BM,LastTerm_BM,Student_Status,FIN221_IND,ACCY200_IND,ACCY201_IND,ACCY202_IND,BADM310_IND,BADM320_IND,GradTerm,BM_Grad_IND,BM_Declaration_IND,BM_Drop_IND,Completion,Core_Completion,Elective_Completion,REG_TERM_str,term_code,term_index,prev_core,prev_elective,prev_total,core_increase,elective_increase,total_increase,took_this_term,core_increase_flag,elective_increase_flag
0,10000197,120218,Sophomore,College of Liberal Arts and Sciences (LAS),,General Curriculum,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120218,120218,0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
2,10000197,120221,Junior,College of Liberal Arts and Sciences (LAS),,Molecular and Cellular Biology,,,,Informatics,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120221,120221,2,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
3,10000197,120228,Junior,College of Liberal Arts and Sciences (LAS),,Molecular and Cellular Biology,,,,Informatics,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120228,120228,5,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
4,10000197,120231,Senior,College of Liberal Arts and Sciences (LAS),,Molecular and Cellular Biology,,,,Informatics,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120231,120231,7,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
5,10000495,120218,Freshman,College of Liberal Arts and Sciences (LAS),,Anthropology,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120218,120218,0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323261,99994330,120258,Junior,College of Applied Health Sciences (AHS),,Speech & Hearing Science,,,,Disability Studies,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120258,120258,20,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
323263,99997995,120248,Freshman,College of Liberal Arts and Sciences (LAS),,LAS - Undeclared,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120248,120248,15,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
323264,99997995,120251,Freshman,College of Liberal Arts and Sciences (LAS),,LAS - Undeclared,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120251,120251,17,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
323265,99997995,120258,Freshman,College of Liberal Arts and Sciences (LAS),,LAS - Undeclared,,,,,,,,NaN,NaN,Other,0,0,0,0,0,0,NaN,2,0,0,0,0,0,120258,120258,20,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False


df_filter['LastTerm_BM'] = df['LastTerm_BM'].replace({
    120215: 120211,
    120210: 120211,
    120217: 120218,
    120225: 120221,
    120220: 120221,
    120227: 120228,
    120235: 120231,
    120230: 120231,
    120237: 120238,
    120245: 120241,
    120240: 120241,
    120247: 120248,
    120255: 120251,
    120250: 120251,
    120257: 120258,
    
})
df_filter['FirstTerm_BM'] = df['FirstTerm_BM'].replace({
    120215: 120211,
    120210: 120211,
    120217: 120218,
    120225: 120221,
    120220: 120221,
    120227: 120228,
    120235: 120231,
    120230: 120231,
    120237: 120238,
    120245: 120241,
    120240: 120241,
    120247: 120248,
    120255: 120251,
    120250: 120251,
    120257: 120258,
    
})

In [470]:
a = df_filter['FirstTerm_BM'].astype(str).str.strip().fillna('')
b = df_filter['REG_TERM'].astype(str).str.strip().fillna('')
a = a.str.replace(r'\.0+$', '', regex=True)
b = b.str.replace(r'\.0+$', '', regex=True)
mask = (df_filter['BM_Declaration_IND'] == 1) & (a != b)

print("Rows that will be changed:", mask.sum())

df_filter.loc[mask, 'BM_Declaration_IND'] = 0


Rows that will be changed: 26463


In [471]:
c = df_filter['LastTerm_BM'].astype(str).str.strip().fillna('')
d = df_filter['REG_TERM'].astype(str).str.strip().fillna('')
c = c.str.replace(r'\.0+$', '', regex=True)
d = d.str.replace(r'\.0+$', '', regex=True)
mask = (df_filter['BM_Drop_IND'] == 1) & (c != d) 

print("Rows that will be changed:", mask.sum())

df_filter.loc[mask, 'BM_Drop_IND'] = 0

Rows that will be changed: 4375


Changed df_filter

In [472]:
declared_students = df_filter[df_filter["BM_Declaration_IND"] == 1]

dropped_students = df_filter[df_filter["BM_Drop_IND"] == 1]

returning = df_filter[(df_filter["FirstTerm_BM"] < df_filter["REG_TERM"]) & (df_filter["FirstTerm_BM"].notna()) & (df_filter["LastTerm_BM"] >= df_filter["REG_TERM"])]

active_df = df_filter[(( df_filter["Student_Status"] == "BM-NEW" ) | ( df_filter["Student_Status"] == "BM-RETURNING" )) ]

taking_df = df_filter[(( df_filter["took_this_term"] == True ))]

active_takedf = df_filter[(( df_filter["Student_Status"] == "BM-NEW" ) | ( df_filter["Student_Status"] == "BM-RETURNING" )) & (( df_filter["took_this_term"] == True ))]

In [473]:
declared_by_semester = declared_students.groupby("REG_TERM").size().reset_index(name='Declared')
declared_by_semester

,REG_TERM,Declared
0,120218,433
1,120221,496
2,120228,518
3,120231,609
4,120238,609
5,120241,552
6,120248,681
7,120251,579
8,120258,333


should be the same

In [474]:
declared_by_semester = df_filter[df_filter["Student_Status"] == "BM-NEW"].groupby("REG_TERM").size().reset_index(name='Declared')
declared_by_semester

,REG_TERM,Declared
0,120218,433
1,120221,496
2,120228,518
3,120231,609
4,120238,609
5,120241,552
6,120248,681
7,120251,579
8,120258,333


In [475]:
by_dropped = (
    dropped_students.groupby("REG_TERM").size().reset_index(name="Dropped")
)
by_dropped 

,REG_TERM,Dropped
0,120218,56
1,120221,58
2,120228,92
3,120231,102
4,120238,109
5,120241,79
6,120248,118
7,120251,70


In [476]:
returning_by_semester = returning.groupby("REG_TERM").size().reset_index(name='Returning')
returning_by_semester

,REG_TERM,Returning
0,120218,1242
1,120221,1500
2,120228,1393
3,120231,1731
4,120238,1687
5,120241,2077
6,120248,1954
7,120251,2384
8,120258,2165


should be same

In [477]:
returning_by_semester = df_filter[df_filter["Student_Status"] == "BM-RETURNING"].groupby("REG_TERM").size().reset_index(name='Returning')
returning_by_semester

,REG_TERM,Returning
0,120218,1242
1,120221,1500
2,120228,1393
3,120231,1731
4,120238,1687
5,120241,2077
6,120248,1954
7,120251,2384
8,120258,2165


Not sure why returning dramtically increases while declared seems relatively stagnant - but this is cross checked some I'm not sure what is the issue here 

In [478]:
active_count = active_df.groupby("REG_TERM").size().reset_index(name='active')
active_count

,REG_TERM,active
0,120218,1675
1,120221,1996
2,120228,1911
3,120231,2340
4,120238,2296
5,120241,2629
6,120248,2635
7,120251,2963
8,120258,2498


Im going to check active and taking vs active estimations - we know active is correct. Based of active - how many people don't take a course at all, take 2 or take 4 cap. 

In [ ]:
activetakescheck = active_df[active]

In [479]:
active_take = active_takedf.groupby("REG_TERM").size().reset_index(name="active and taking")
active_take

,REG_TERM,active and taking
0,120218,1072
1,120221,1079
2,120228,823
3,120231,1208
4,120238,1003
5,120241,1416
6,120248,1179
7,120251,1599
8,120258,1225


In [480]:
completed = df_filter[df_filter["BM_Grad_IND"] == 1].groupby("REG_TERM").size().reset_index(name="Completed")

completed

,REG_TERM,Completed
0,120218,92
1,120221,481
2,120228,70
3,120231,489
4,120238,78
5,120241,513
6,120248,109
7,120251,644


In [481]:
droprate = (by_dropped["Dropped"] / completed["Completed"].shift()).to_frame(name="drop_rate")
droprate

,drop_rate
0,NaN
1,0.630435
2,0.191268
3,1.457143
4,0.222904
5,1.012821
6,0.230019
7,0.642202


In [482]:
size_by_term = df_filter.groupby("REG_TERM").size().reset_index(name="size_by_term")

In [483]:
decl_rate = (declared_by_semester["Declared"] / (size_by_term["size_by_term"].shift() )).to_frame(name="Declared Rate")
decl_rate

,Declared Rate
0,NaN
1,0.015674
2,0.017098
3,0.018957
4,0.019860
5,0.016842
6,0.021531
7,0.016810
8,0.010043


In [484]:
lag_active = (active_count["active"].shift()).to_frame(name="lag_active")
lag_active

,lag_active
0,NaN
1,1675.0
2,1996.0
3,1911.0
4,2340.0
5,2296.0
6,2629.0
7,2635.0
8,2963.0


In [485]:
declared_by_college = (df_filter[df_filter["BM_Declaration_IND"] == 1].groupby("COLL1").size()).to_frame(name="Declared_by_College")

In [486]:
df_sorted = df_filter.sort_values("REG_TERM")
college_counts = (
    df_sorted.groupby(["REG_TERM", "COLL1"])["ID"]
    .nunique()   # counts unique students per term-college
    .reset_index(name="number_of_students_at_college")
)



In [487]:
term_totals = college_counts.groupby("REG_TERM")["number_of_students_at_college"].sum().reset_index(name="total_number_of_college")
term_totals

,REG_TERM,total_number_of_college
0,120218,31644
1,120221,30295
2,120228,32120
3,120231,30661
4,120238,32772
5,120241,31626
6,120248,34440
7,120251,33154
8,120258,35769


In [488]:
college_counts = college_counts.merge(term_totals, on="REG_TERM", how="left")


In [489]:
college_counts["percent"] = college_counts["number_of_students_at_college"] / college_counts["total_number_of_college"]



In [490]:
declared_by_semestercol = df_sorted[df_sorted["BM_Declaration_IND"] == 1].groupby(["REG_TERM", "COLL1"]).size().to_frame("declared_by_semester_college")


In [491]:
by_droppedcol = df_sorted[df_sorted["BM_Drop_IND"] == 1].groupby(["REG_TERM", "COLL1"]).size().to_frame("dropped_college_semester")


In [492]:
returningcol = returning.groupby(["REG_TERM", "COLL1"]).size().to_frame(name="returning_semester_college")


In [493]:
completedcol = df_sorted[df_sorted["BM_Grad_IND"] == 1].groupby(["REG_TERM", "COLL1"]).size().to_frame(name="completed_semester_college")



In [494]:
activecol = declared_by_semestercol["declared_by_semester_college"] + returningcol["returning_semester_college"] 

activecol

activecol_df = activecol.reset_index(name="actives by college")


In [495]:
college_counts = college_counts.merge(
    activecol_df, on=["REG_TERM", "COLL1"], how="left"
)


In [496]:
avg_core_completion = df_filter[df_filter["BM_Declaration_IND"] == 1].groupby("REG_TERM")["Core_Completion"].mean().reset_index(name="mean_core_completion")

In [497]:
avg_elective_completion =  df_filter[df_filter["BM_Declaration_IND"] == 1].groupby("REG_TERM")["Elective_Completion"].mean().reset_index(name="mean_elective_completion")

In [498]:
colleges_to_include = ['College of Agricultural, Consumer & Environmental Sciences (ACES)', 
                       'College of Liberal Arts and Sciences (LAS)', 
                       'Grainger College of Engineering (GCOE)']

college_subset = college_counts[college_counts['COLL1'].isin(colleges_to_include)]

# Pivot to get each college's percentage as a separate column
college_percentages = college_subset.pivot_table(
    index='REG_TERM',
    columns='COLL1',
    values='percent',
    aggfunc='first'
)

# Rename columns to be more descriptive (shortened for readability)
college_percentages.columns = ['ACES_percent', 'LAS_percent', 'GCOE_percent']

# Reset index to make REG_TERM a column
college_percentages = college_percentages.reset_index()

In [499]:
result = pd.concat([
    declared_by_semester,by_dropped["Dropped"],
    completed["Completed"],
    active_count["active"],
    active_take["active and taking"],
    droprate["drop_rate"],
    decl_rate["Declared Rate"],lag_active["lag_active"],
    term_totals["total_number_of_college"],
    avg_core_completion["mean_core_completion"],
    avg_elective_completion["mean_elective_completion"],
    college_percentages
    ],axis=1)
result
result = result.loc[:, ~result.columns.duplicated()]
result

,REG_TERM,Declared,Dropped,Completed,active,active and taking,drop_rate,Declared Rate,lag_active,total_number_of_college,mean_core_completion,mean_elective_completion,ACES_percent,LAS_percent,GCOE_percent
0,120218,433,56.0,92.0,1675,1072,NaN,NaN,NaN,31644,0.427252,0.076212,0.087884,0.378871,0.264916
1,120221,496,58.0,481.0,1996,1079,0.630435,0.015674,1675.0,30295,0.231855,0.040323,0.088067,0.381878,0.266843
2,120228,518,92.0,70.0,1911,823,0.191268,0.017098,1996.0,32120,0.254826,0.028958,0.082130,0.374595,0.274875
3,120231,609,102.0,489.0,2340,1208,1.457143,0.018957,1911.0,30661,0.221675,0.036125,0.083950,0.376700,0.276671
4,120238,609,109.0,78.0,2296,1003,0.222904,0.019860,2340.0,32772,0.206897,0.029557,0.080892,0.372818,0.281033
5,120241,552,79.0,513.0,2629,1416,1.012821,0.016842,2296.0,31626,0.177536,0.030797,0.083318,0.376431,0.278410
6,120248,681,118.0,109.0,2635,1179,0.230019,0.021531,2629.0,34440,0.152717,0.017621,0.083972,0.367625,0.285918
7,120251,579,70.0,644.0,2963,1599,0.642202,0.016810,2635.0,33154,0.117444,0.020725,0.086113,0.371901,0.281897
8,120258,333,NaN,NaN,2498,1225,NaN,0.010043,2963.0,35769,0.144144,0.006006,0.086164,0.362101,0.289692


In [500]:
college_counts = college_counts.merge(
    activecol_df, on=["REG_TERM", "COLL1"], how="left"
)
college_counts

,REG_TERM,COLL1,number_of_students_at_college,total_number_of_college,percent,actives by college_x,actives by college_y
0,120218,Center Innov in Teach Learn,109,31644,0.003445,NaN,NaN
1,120218,"College of Agricultural, Consumer & Environmen...",2781,31644,0.087884,236.0,236.0
2,120218,College of Applied Health Sciences (AHS),2145,31644,0.067785,130.0,130.0
3,120218,College of Education (EDUC),660,31644,0.020857,3.0,3.0
4,120218,College of Fine and Applied Arts (FAA),1765,31644,0.055777,87.0,87.0
...,...,...,...,...,...,...,...
102,120258,Division of Exploratory Studies (Undeclared),1751,35769,0.048953,38.0,38.0
103,120258,Grainger College of Engineering (GCOE),10362,35769,0.289692,457.0,457.0
104,120258,Provost & VC Acad Affairs,117,35769,0.003271,NaN,NaN
105,120258,School of Information Sciences (iSchool),902,35769,0.025217,82.0,82.0


In [501]:
final_df = result.merge(
    college_counts, 
    left_on='REG_TERM',  # or whatever the term column is called in result
    right_on='REG_TERM', 
    how='outer'
)
final_df

,REG_TERM,Declared,Dropped,Completed,active,active and taking,drop_rate,Declared Rate,lag_active,total_number_of_college_x,mean_core_completion,mean_elective_completion,ACES_percent,LAS_percent,GCOE_percent,COLL1,number_of_students_at_college,total_number_of_college_y,percent,actives by college_x,actives by college_y
0,120218,433,56.0,92.0,1675,1072,NaN,NaN,NaN,31644,0.427252,0.076212,0.087884,0.378871,0.264916,Center Innov in Teach Learn,109,31644,0.003445,NaN,NaN
1,120218,433,56.0,92.0,1675,1072,NaN,NaN,NaN,31644,0.427252,0.076212,0.087884,0.378871,0.264916,"College of Agricultural, Consumer & Environmen...",2781,31644,0.087884,236.0,236.0
2,120218,433,56.0,92.0,1675,1072,NaN,NaN,NaN,31644,0.427252,0.076212,0.087884,0.378871,0.264916,College of Applied Health Sciences (AHS),2145,31644,0.067785,130.0,130.0
3,120218,433,56.0,92.0,1675,1072,NaN,NaN,NaN,31644,0.427252,0.076212,0.087884,0.378871,0.264916,College of Education (EDUC),660,31644,0.020857,3.0,3.0
4,120218,433,56.0,92.0,1675,1072,NaN,NaN,NaN,31644,0.427252,0.076212,0.087884,0.378871,0.264916,College of Fine and Applied Arts (FAA),1765,31644,0.055777,87.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,120258,333,NaN,NaN,2498,1225,NaN,0.010043,2963.0,35769,0.144144,0.006006,0.086164,0.362101,0.289692,Division of Exploratory Studies (Undeclared),1751,35769,0.048953,38.0,38.0
103,120258,333,NaN,NaN,2498,1225,NaN,0.010043,2963.0,35769,0.144144,0.006006,0.086164,0.362101,0.289692,Grainger College of Engineering (GCOE),10362,35769,0.289692,457.0,457.0
104,120258,333,NaN,NaN,2498,1225,NaN,0.010043,2963.0,35769,0.144144,0.006006,0.086164,0.362101,0.289692,Provost & VC Acad Affairs,117,35769,0.003271,NaN,NaN
105,120258,333,NaN,NaN,2498,1225,NaN,0.010043,2963.0,35769,0.144144,0.006006,0.086164,0.362101,0.289692,School of Information Sciences (iSchool),902,35769,0.025217,82.0,82.0


In [ ]:

result

,REG_TERM,Declared,Dropped,Completed,active,active and taking,drop_rate,Declared Rate,lag_active,total_number_of_college,mean_core_completion,mean_elective_completion,ACES_percent,LAS_percent,GCOE_percent
0,120218,433,56.0,92.0,1675,1072,NaN,NaN,NaN,31644,0.427252,0.076212,0.087884,0.378871,0.264916
1,120221,496,58.0,481.0,1996,1079,0.630435,0.015674,1675.0,30295,0.231855,0.040323,0.088067,0.381878,0.266843
2,120228,518,92.0,70.0,1911,823,0.191268,0.017098,1996.0,32120,0.254826,0.028958,0.082130,0.374595,0.274875
3,120231,609,102.0,489.0,2340,1208,1.457143,0.018957,1911.0,30661,0.221675,0.036125,0.083950,0.376700,0.276671
4,120238,609,109.0,78.0,2296,1003,0.222904,0.019860,2340.0,32772,0.206897,0.029557,0.080892,0.372818,0.281033
5,120241,552,79.0,513.0,2629,1416,1.012821,0.016842,2296.0,31626,0.177536,0.030797,0.083318,0.376431,0.278410
6,120248,681,118.0,109.0,2635,1179,0.230019,0.021531,2629.0,34440,0.152717,0.017621,0.083972,0.367625,0.285918
7,120251,579,70.0,644.0,2963,1599,0.642202,0.016810,2635.0,33154,0.117444,0.020725,0.086113,0.371901,0.281897
8,120258,333,NaN,NaN,2498,1225,NaN,0.010043,2963.0,35769,0.144144,0.006006,0.086164,0.362101,0.289692


In [504]:
result.to_csv('term_level_summary-alex_clean.csv', index=False)